<a href="https://colab.research.google.com/github/Alexher90/An-lisis-de-Datos-2/blob/master/Solucion_Semana14_EvaluacionModelos_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MIIA-4203 MODELOS AVANZADOS PARA ANÁLISIS DE DATOS II


# Evaluacion de recomendaciones

## Actividad 14

### Profesor: Camilo Franco (c.franco31@uniandes.edu.co)

En este cuaderno vamos a estudiar cómo evaluar las recomendaciones que obtenemos de nuestros algoritmos, enfocándonos en la *evaluación de rankings*. AL mismo tiempo, seguiremos trabajndo con los métodos de filtrado colaborativo, centrándonos en los métodos de *Baseline* o de promedios corregidos, de descomposición matricial mediante valores singulares y la factorizacion matricial no-negativa, y de vecindades por los k-vecinos más cercanos.

De este modo, la evaluación de los rankings o de las listas de recomendación en base a las primeras N recomendaciones nos permiten evaluar métodos sólo a partir de su output. Sin importar si contamos con ratings explícitos o implícitos, podemos centrarnos en evaluar qué tan acertado es el *ranking* de los items, o esas recomendaciones sobre las que los usuarios toman sus decisiones.


**Nombres: Alexander Hernández Páez (200920588), Alexander Camargo (200911325), Juan David Cortés (201728568), Wilson Felipe González (200924943).**

## 1.  Contrucción de algoritmos

Podemos evaluar los sistemas de recomendación de acuerdo con el ranking que asignan sobre los items. Entonces nos interesa medir su nivel de acierto sobre un *Top-N* de items. Piense en la lista que ofrece un buscador o en un portal como youtube donde la recomendación de videos es una lista en orden de preferencia.  

Puede ver el siguiente video sobre un sistema de recomendacion de este tipo en Amazon: https://www.youtube.com/watch?v=EeXBdQYs0CQ

A continuación veamos algunos ejemplos sobre las recomendaciones que se pueden hacer en forma de ranking de preferencias. 

Finalmente, si quiere investigar sobre cómo calcular diferentes métricas como Precision@k and Recall@k, puede ver: https://surprise.readthedocs.io/en/stable/FAQ.html

Implementemos una función `Top_N` que recibe las predicciones del sistema, el numero de recomendaciones a ofrecer y el rating mínimo a tener en cuenta.

A continuación entrenemos un algoritmo *BaselineOnly* que ofrezca las 10 mejores recomendaciones para cada usuario.

Primero importemos las bibliotecas y funciones que vamos a utilizar:


In [3]:
pip install surprise

  Using cached https://files.pythonhosted.org/packages/61/de/e5cba8682201fcf9c3719a6fdda95693468ed061945493dea2dd37c5618b/surprise-0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/97/37/5d334adaf5ddd65da99fc65f6507e0e4599d092ba048f4302fe8775619e8/scikit-surprise-1.1.1.tar.gz
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670957 sha256=9a10d2642d30f268dc4c1a11be187c82e9ffc2e6759862bfefb54410c9018deb
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [4]:
import numpy as np
import pandas as pd
from surprise import Reader, Dataset, BaselineOnly, accuracy
from surprise.model_selection import LeaveOneOut, train_test_split
from collections import defaultdict

Carguemos los datos de las películas y las preferencias:

In [5]:
pelis = pd.read_csv('movies_metadata.csv', low_memory=False)

df = pd.read_csv('ratings_small.csv')
df.drop(['timestamp'], axis=1, inplace=True)
df

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


Veamos las preferencias por usuario:

In [6]:
df[df['userId'] == 2]

,userId,movieId,rating
20,2,10,4.0
21,2,17,5.0
22,2,39,5.0
23,2,47,4.0
24,2,50,4.0
...,...,...,...
91,2,592,5.0
92,2,593,3.0
93,2,616,3.0
94,2,661,4.0


Filtremos los datos para quedarnos con los usuarios y peliculas con un mínimo de entradas:

In [7]:
min_p_ratings = 5
filter_p = df['movieId'].value_counts() > min_p_ratings
filter_p = filter_p[filter_p].index.tolist()

min_u_ratings = 30
filter_u = df['userId'].value_counts() > min_u_ratings
filter_u = filter_u[filter_u].index.tolist()

df_nuevo = df[(df['movieId'].isin(filter_p)) & (df['userId'].isin(filter_u))]
print('Los datos originales tienen tamaño:\t{}'.format(df.shape))
print('Los nuevo datos tienen tamaño:\t{}'.format(df_nuevo.shape))

Los datos originales tienen tamaño:	(100004, 3)
Los nuevo datos tienen tamaño:	(85239, 3)


Declaremos los datos para trabajarlos con SurPRISE:

In [8]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df_nuevo[['userId', 'movieId', 'rating']], reader)

data

Especifiquemos las opciones para la implementacion del algoritmo del *BaselineOnly*:

In [9]:
bsl_options = {'method': 'als',
               'n_epochs': 20,
               'reg_u': 12, 
               'reg_i': 5  
               }

Podemos entrenar el algoritmo sobre el 75% de los datos con rating observado y lo probamos sobre la muestra sobrante:

In [10]:
trainSet, testSet = train_test_split(data, test_size=.25, random_state=0)
algoritmo = BaselineOnly(bsl_options=bsl_options)
algoritmo.fit(trainSet)
preds = algoritmo.test(testSet)

Estimating biases using als...


O podemos construirlo mediante validacion cruzada. Exploremos el uso del LOOCV:

In [11]:
LOOCV = LeaveOneOut(n_splits=1, random_state=1)
algoritmo2 = BaselineOnly(bsl_options=bsl_options)

for trainSet, testSet in LOOCV.split(data):
    # Entrenamos el modelo en entrenamiento
    algoritmo2.fit(trainSet)
    # Predecimos
    predV = algoritmo2.test(testSet)

Estimating biases using als...


### Pregunta 1

Cuál es el RMSE resultante según el método de construcción del algoritmo del BaselineOnly?

In [12]:
import datetime

In [13]:
trainset, testset = train_test_split(data, test_size=0.3)
Base = BaselineOnly(bsl_options=bsl_options)

tiempo = datetime.datetime.now()
print('\nTermina: ', tiempo)

print("\nBaselineOnly: ")
tiempo = datetime.datetime.now()
print('\nInicia el entrenamiento y prueba: ', tiempo)

predBase = Base.fit(trainset).test(testset)
print("\nRMSE del BaselineOnly: ", accuracy.rmse(predBase))

tiempo = datetime.datetime.now()
print('\nTermina: ', tiempo)


Termina:  2020-11-29 23:29:26.073200

BaselineOnly: 

Inicia el entrenamiento y prueba:  2020-11-29 23:29:26.074108
Estimating biases using als...
RMSE: 0.8753

RMSE del BaselineOnly:  0.8753021305904519

Termina:  2020-11-29 23:29:26.492061


### Ejercicio 2

Encuentre un algoritmo distinto con resultados al menos tan buenos como los obtenidos por el *BaselineOnly*

In [14]:
import surprise
from surprise import Reader
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise.model_selection import LeaveOneOut

In [15]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=10, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.856090,341.718077,3.287392
KNNBaseline,0.861302,0.346738,1.082578
SVD,0.869999,5.008862,0.066101
BaselineOnly,0.871546,0.217571,0.061418
KNNWithZScore,0.874141,0.205626,0.956659
KNNWithMeans,0.876058,0.160673,0.884197
SlopeOne,0.882668,1.812730,2.611692
NMF,0.901271,5.183552,0.067285
KNNBasic,0.921880,0.140622,0.830761


**De acuerdo con los resultados del modelo anterior, podemos ver que el modelo SVDpp fue el que tuvo el mejor rendimiento en términos del RMSE. Adicionalmente, podemos ver que los modelos KKNBaseline y SVD tuvieron un mejor rendimiento que el Baseline Only.**

## 2. Evaluacion de rankings

Construyamos una función para quedarnos con las N recomendaciones más atractivas para cada usuario:

In [16]:
def Top_N(preds, N, min_r):
    topN = defaultdict(list)
    for uid, iid, r_ui, est, _ in preds:
        if (est >= min_r):
            topN[int(uid)].append((int(iid), est))

    for uid, ratings in topN.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        topN[int(uid)] = ratings[:N]

    return topN

Veamos qué tan bien le va a nuestro algoritmo del *BaselineOnly*. La siguiente celda toma el conjunto de entrenamiento de la implementacion del LOOCV y predice para los ratings sobrantes:

In [17]:
# Obtenemos las predicciones que no están en entrenamiento
X_Prueba = trainSet.build_anti_testset() #observaciones sin rating
Preds_T = algoritmo.test(X_Prueba)
# Calculamos las I recomendaciones para cada usuario
topN_pred = Top_N(Preds_T, N=10, min_r=3.0)

Veamos las recomendaciones para un usuario:

In [18]:
#topN_pred.keys()
usuario=40
topN_pred[usuario]

[(969, 4.829882881081904),
 (318, 4.814097149422216),
 (3462, 4.778969585834668),
 (1221, 4.755216895401907),
 (858, 4.740759357195653),
 (3088, 4.719898252433616),
 (1172, 4.675544584905746),
 (1193, 4.66348839144212),
 (923, 4.6603933676684655),
 (1945, 4.658622202021493)]

Veamos los titulos de las peliculas que tenemos en nuestra base original (pelis):

In [19]:
for i in range(10):
    print(np.squeeze(pelis[pelis.id == str(topN_pred[usuario][i][0])]['original_title']))

Series([], Name: original_title, dtype: object)
The Million Dollar Hotel
Series([], Name: original_title, dtype: object)
Series([], Name: original_title, dtype: object)
Sleepless in Seattle
My Darling Clementine
Series([], Name: original_title, dtype: object)
Series([], Name: original_title, dtype: object)
Dawn of the Dead
Nell


### 2.1 Tasa de aciertos

Para evaluar nuestro recomendador, vemos la tasa de acierto. Esto es, si un usuario calificó entre sus primeras N preferencias una de las peliculas recomendadas, entonces lo consideramos un acierto.

Entonces computamos la tasa de acierto para cada usuario:

- Encontramos todos los items en la historia del usuario en los datos de entrenamiento.
- Quitamos uno de esos items (algo como *Leave-One-Out cross-validation*).
- Usamos los demás items para entrenar el recomendador y pedimos las N recomendaciones.
- Si el item que dejamos fuera aparece en las top-N recomendaciones entonces lo consideramos un acierto. 


In [20]:
def TasaAcierto(topN_pred, predV):
    aciertos = 0
    total = 0

 # Para cada observacion dejada por fuera
    for obs_out in predV:
        userID = obs_out[0]
        movieID_V = obs_out[1]
        # Verficamos si se encuentra en el top-N
        acierto = False
        for movieID, predRating in topN_pred[int(userID)]:
            if (int(movieID_V) == int(movieID)):
                acierto = True
                break
        if (acierto) :
            aciertos += 1

        total += 1

    # Calculamos la tasa de aciertos
    return aciertos/total
print("\nTasa de aciertos: ", TasaAcierto(topN_pred, predV))


Tasa de aciertos:  0.020146520146520148


Deseariamos que la tasa fuera más alta. Los resultados se pueden deber a que no contamos con suficientes datos para todos los usuarios.

### 2.2 Tasa de aciertos por nivel de rating

También podemos discriminar por niveles del rating:

In [21]:
def TasaAcierto_R(topN_pred, predV):
    aciertos = defaultdict(float)
    total = defaultdict(float)
    # Para cada rating por fuera de la muestra
    for userID, movieID_V, ratings, est_r, _ in predV:
        # Verficamos si se encuentra en el top-N
        acierto = False
        for movieID, pred_r in topN_pred[int(userID)]:
            if (int(movieID_V) == movieID):
                acierto = True
                break
        if (acierto) :
            aciertos[ratings] += 1
        total[ratings] += 1

    # Calculamos la tasa de aciertos
    for rating in sorted(aciertos.keys()):
        print(rating, aciertos[rating] / total[rating])
print("Tasa de aciertos por nivel de rating: ")
TasaAcierto_R(topN_pred, predV)

Tasa de aciertos por nivel de rating: 
3.5 0.020833333333333332
4.0 0.012345679012345678
4.5 0.022222222222222223
5.0 0.06666666666666667


La tasa mejora para ratings de 5, lo cual es una buena señal pues nos interesa que el sistema acierte más con las peliculas que los usuarios más prefieren.

### 2.3 Tasa de acierto acumulativa

Calculemos la tasa de acierto para ratings mayores que 4.0:

In [22]:
def TasaAcierto_Acum(topN_pred, predV, umbral):
    aciertos = 0
    total = 0
    # Para cada rating por fuera de la muestra
    for userID, movieID_V, ratings, est_r, _ in predV:
        # Nos fijamos solo en lo que los usuarios prefieren más
        if (ratings >= umbral):
            # Verficamos si se encuentra en el top-N
            acierto = False
            for movieID, pred_r in topN_pred[int(userID)]:
                if (int(movieID_V) == movieID):
                    acierto = True
                    break
            if (acierto) :
                aciertos += 1
            total += 1

        # Calculamos la precision global
    return aciertos/total
print("Tasa de aciertos acumulada (rating >= 4): ", 
      TasaAcierto_Acum(topN_pred, predV, 4.0))

Tasa de aciertos acumulada (rating >= 4):  0.03205128205128205


### 2.4 Tasa de aciertos promedio recíproca 

Otra metrica relevante es la tasa de aciertos promedio recíproca.

Tomamos en cuenta donde ocurre el primer resultado relevante. Es decir, si la primera recomendación pertenece al primer lugar la tasa recíproca es de 1; si aparece en segundo lugar es de 1/2; en tercer lugar es de 1/3; etc.


In [23]:
def TasaAcierto_Recip(topN_pred, predV):
    suma = 0
    total = 0
    # Para cada rating por fuera de la muestra
    for userID, movieID_V, ratings, est_r, _ in predV:
        # Verficamos si se encuentra en el top-N
        aciertoRank = 0
        rank = 0
        for movieID, pred_r in topN_pred[int(userID)]:
            rank = rank + 1
            if (int(movieID_V) == movieID):
                aciertoRank = rank
                break
        if (aciertoRank > 0) :
                suma += 1.0 / aciertoRank

        total += 1

    return suma / total

print("Tasa reciproca media de aciertos : ", 
      TasaAcierto_Recip(topN_pred, predV,))

Tasa reciproca media de aciertos :  0.00572562358276644


Una tasa reciproca media de aciertos de 0.0057 nos dice que en promedio la tasa reciproca es de 1/133. De nuevo, este valor tan bajo se puede deber a que en muchos casos el algoritmo simplemente no logra recomendar las peliculas más preferidas de ciertos usuarios con poca información.

### Ejercicio 3

Evalúe su algoritmo propuesto en el Ejercicio 2 de acuerdo con la tasa de aciertos promedio recíproca. Analice si su algoritmo mejora las recomendaciones del BaselineOnly. El grupo que logre la mejor tasa se ganará un bono en las Actividades.

In [59]:
bsl_options = {'method': 'als',
               'n_epochs': 20,
               'reg_u': 12, 
               'reg_i': 5  
               }

sim_options = {'name': 'pearson_baseline',
               'user_based': False  # calcula similitudes entre items
               }

In [60]:
SVD = SVDpp(random_state=0)

print("\nSVDpp: ")
tiempo = datetime.datetime.now()
print('\nInicia el entrenamiento y prueba: ', tiempo)

predSVD = SVD.fit(trainset).test(testset)
print("RMSE del SVDpp: ", accuracy.rmse(predSVD))

tiempo = datetime.datetime.now()
print('\nTermina: ', tiempo)


SVDpp: 

Inicia el entrenamiento y prueba:  2020-11-30 02:21:59.414990
RMSE: 0.8689
RMSE del SVDpp:  0.8689042285037836

Termina:  2020-11-30 02:25:36.038776


In [61]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)
algoritmo = SVD
algoritmo.fit(trainset)
preds = algoritmo.test(testset)

In [62]:
# Obtenemos las predicciones que no están en entrenamiento
X_Prueba = trainset.build_anti_testset() #observaciones sin rating
Preds_T = algoritmo.test(X_Prueba)
# Calculamos las I recomendaciones para cada usuario
topN_pred = Top_N(Preds_T, N=10, min_r=3.0)

In [63]:
def TasaAcierto_Recip(topN_pred, predSVD):
    suma = 0
    total = 0
    # Para cada rating por fuera de la muestra
    for userID, movieID_V, ratings, est_r, _ in predSVD:
        # Verficamos si se encuentra en el top-N
        aciertoRank = 0
        rank = 0
        for movieID, pred_r in topN_pred[int(userID)]:
            rank = rank + 1
            if (int(movieID_V) == movieID):
                aciertoRank = rank
                break
        if (aciertoRank > 0) :
                suma += 1.0 / aciertoRank

        total += 1

    return suma / total

print("Tasa reciproca media de aciertos : ", 
      TasaAcierto_Recip(topN_pred, predSVD,))

Tasa reciproca media de aciertos :  0.0016631068455329374


**El grupo de trabajo implementó el algoritmo de factor latente SVD, el cual fue el algoritmo que mejor tuvo rendimiento en el ejercicio anterior. Sin embargo, este no mejoró la Tasa de Aciertos Promedio Recíproca, por lo que fue necesario implementar el modelo variando el algoritmo como se muestra a continuación:**

**Modelo con KNN Baseline:**

In [91]:
bsl_options = {'method': 'als',
               'n_epochs': 20,
               'reg_u': 50, 
               'reg_i': 35  
               }

sim_options = {'name': 'pearson_baseline',
               'user_based': False  # calcula similitudes entre items
               }

In [92]:
trainset, testset = train_test_split(data, test_size=0.3)

KNN = KNNBaseline(bsl_options=bsl_options, sim_options=sim_options)
Base = KNNBaseline(k=51, bsl_options=bsl_options)

print("\nKNNBaseline: ")
tiempo = datetime.datetime.now()
print('\nInicia el entrenamiento y prueba: ', tiempo)

predKNN = KNN.fit(trainset).test(testset)
print("\nFCP del KNNBaseline: ", accuracy.rmse(predKNN))

tiempo = datetime.datetime.now()
print('\nTermina: ', tiempo)


KNNBaseline: 

Inicia el entrenamiento y prueba:  2020-11-30 03:02:33.797685
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8703

FCP del KNNBaseline:  0.8703443780272654

Termina:  2020-11-30 03:02:45.346987


In [93]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)
algoritmo = KNN
algoritmo.fit(trainset)
preds = algoritmo.test(testset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [94]:
# Obtenemos las predicciones que no están en entrenamiento
X_Prueba = trainset.build_anti_testset() #observaciones sin rating
Preds_T = algoritmo.test(X_Prueba)
# Calculamos las I recomendaciones para cada usuario
topN_pred = Top_N(Preds_T, N=10, min_r=3.0)

In [95]:
def TasaAcierto_Recip(topN_pred, predKNN):
    suma = 0
    total = 0
    # Para cada rating por fuera de la muestra
    for userID, movieID_V, ratings, est_r, _ in predKNN:
        # Verficamos si se encuentra en el top-Nd
        aciertoRank = 0
        rank = 0
        for movieID, pred_r in topN_pred[int(userID)]:
            rank = rank + 1
            if (int(movieID_V) == movieID):
                aciertoRank = rank
                break
        if (aciertoRank > 0) :
                suma += 1.0 / aciertoRank

        total += 1

    return suma / total

print("Tasa reciproca media de aciertos : ", 
      TasaAcierto_Recip(topN_pred, predKNN,))

Tasa reciproca media de aciertos :  0.0018019150410046726


**Al no haber obtenido un resultado significativamente mejor que el anterior modelo, el grupo de trabajo recurrió a implementarlo por medio del algoritmo Baseline Only, manteniendo los parámetros del modelo anterior:**

**Prueba con Baseline Only:**

In [96]:
import surprise

from surprise import Reader
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise.model_selection import LeaveOneOut

In [97]:
bsl_options = {'method': 'als',
               'n_epochs': 20,
               'reg_u': 50, 
               'reg_i': 35  
               }

sim_options = {'name': 'pearson_baseline',
               'user_based': False  # calcula similitudes entre items
               }

In [99]:
trainset, testset = train_test_split(data, test_size=0.3)

print("\nBaselineOnly: ")
tiempo = datetime.datetime.now()
print('\nInicia el entrenamiento y prueba: ', tiempo)

predBase = BaselineOnly(bsl_options=bsl_options)
Base.fit(trainset).test(testset)
#print("\nRMSE del BaselineOnly: ", accuracy.rmse(predBase))

tiempo = datetime.datetime.now()
print('\nTermina: ', tiempo)


BaselineOnly: 

Inicia el entrenamiento y prueba:  2020-11-30 03:07:48.714756
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.

Termina:  2020-11-30 03:07:52.025432


In [100]:
# Obtenemos las predicciones que no están en entrenamiento
X_Prueba = trainset.build_anti_testset() #observaciones sin rating
Preds_T = algoritmo.test(X_Prueba)
# Calculamos las I recomendaciones para cada usuario
topN_pred = Top_N(Preds_T, N=10, min_r=3.0)

In [101]:
def TasaAcierto_Recip(topN_pred, predKNN):
    suma = 0
    total = 0
    # Para cada rating por fuera de la muestra
    for userID, movieID_V, ratings, est_r, _ in predKNN:
        # Verficamos si se encuentra en el top-Nd
        aciertoRank = 0
        rank = 0
        for movieID, pred_r in topN_pred[int(userID)]:
            rank = rank + 1
            if (int(movieID_V) == movieID):
                aciertoRank = rank
                break
        if (aciertoRank > 0) :
                suma += 1.0 / aciertoRank

        total += 1

    return suma / total

print("Tasa reciproca media de aciertos : ", 
      TasaAcierto_Recip(topN_pred, predKNN,))

Tasa reciproca media de aciertos :  0.006775112412137279


**Según el resultado anterior, podemos evidenciar que con el modelo probado con BaseLine Only, el resultado de la Tasa de Aciertos Promedio Recíproca crece significativamente, superando el índice inicial de 0.0057, siendo éste en 0.0067.**